## Rename All SceneViewing Folders and Files From All Blocks to Sourcedata Fromat 

VERSION: `Rename_SceneViewing_All_Blocks_Sourcedata`

Renames files and session folders from every block into the sourcedata format

This script will read the edf to extract date from the file itself and will determine the ID based on the Participant ID folder. This script should only be run after all the files have been sorted into their respective folders. If dates cannot be extracted from the file directly, it will prompt the user to enter the information manually.

*Note that the script determines ID based on the folders instead of the files inside because any naming errors are only corrected at the folder level (does not include the files inside) post data collection.

Script checks for redundancies only at the session folder level to avoid long run time. This means if a session folder is just numbers, it will assume that it has been renamed and skip it. 

In [ ]:
#!/bin/bash

extract_date() {
    local edf_file=$1
    local date_info=$(strings "$edf_file" | grep "^DATE:" | awk -F "DATE:" '{print $2}')
    local month=$(echo "$date_info" | awk '{print $2}')
    local day=$(echo "$date_info" | awk '{print $3}')
    local year=$(echo "$date_info" | awk '{print $NF}')
    local month_num=$(date -d "$month 1 2000" +%m 2> /dev/null)
    
    if [ -z "$month_num" ]; then
        echo "Error: Unable to extract date from $edf_file."
        return 1
    fi

    month_num=$((10#$month_num))
    day=$((10#$day))

    printf "%4d%02d%02d" "$year" "$month_num" "$day"
}

extract_block_number() {
    local dat_file=$1
    local block_type=$2

    if [[ "$block_type" == "Encoding" ]]; then
        echo "$dat_file" | grep -oP 'FinalR\K\d+'
    elif [[ "$block_type" == "Retrieval" ]]; then
        echo "$dat_file" | grep -oP 'Block_\K\d+'
    fi
}

rename_files() {
    local block_type=$1
    local id=$2
    local date=$3
    local block=$4
    local session_dir=$5

    if [[ "$block_type" == "Encoding" ]]; then
        local edf_file=$(find "$session_dir" -type f -name "*.edf")
        local trial_dat_file=$(find "$session_dir" -type f -name "*Trial*.dat")
        local first_trial_dat_file=$(find "$session_dir" -type f -name "*firsttrialFixation*.dat")
        local log_file=$(find "$session_dir" -type f -name "*.log")

        [ -f "$edf_file" ] && mv "$edf_file" "${session_dir}/sub-${id}_date-${date}_proj-sceneviewing_task-study_run-${block}_eyetrack.edf"
        [ -f "$trial_dat_file" ] && mv "$trial_dat_file" "${session_dir}/sub-${id}_date-${date}_proj-sceneviewing_task-study_run-${block}_eyetrack.dat"
        [ -f "$first_trial_dat_file" ] && mv "$first_trial_dat_file" "${session_dir}/sub-${id}_date-${date}_proj-sceneviewing_task-study_run-${block}_subset-firstfixation_eyetrack.dat"
        [ -f "$log_file" ] && mv "$log_file" "${session_dir}/sub-${id}_date-${date}_proj-sceneviewing_task-study_run-${block}_eyetrack.log"

    elif [[ "$block_type" == "Retrieval" ]]; then
        local edf_file=$(find "$session_dir" -type f -name "*.edf")
        local trial_dat_file=$(find "$session_dir" -type f -name "*Trial*.dat")
        local txt_file=$(find "$session_dir" -type f -name "*.txt")
        local warning_log_file=$(find "$session_dir" -type f -name "warning.log")
        local eb_messages_log_file=$(find "$session_dir" -type f -name "eb_messages.log")

        [ -f "$edf_file" ] && mv "$edf_file" "${session_dir}/sub-${id}_${date}_proj-sceneviewing_task-recognition_run-${block}_eyetrack.edf"
        [ -f "$trial_dat_file" ] && mv "$trial_dat_file" "${session_dir}/sub-${id}_${date}_proj-sceneviewing_task-recognition_run-${block}_eyetrack.dat"
        [ -f "$txt_file" ] && mv "$txt_file" "${session_dir}/sub-${id}_${date}_scene_recognition_run${block}.txt"
        [ -f "$warning_log_file" ] && mv "$warning_log_file" "${session_dir}/sub-${id}_${date}_proj-sceneviewing_task-recognition_run${block}_log-warning_eyetrack.log"
        [ -f "$eb_messages_log_file" ] && mv "$eb_messages_log_file" "${session_dir}/sub-${id}_${date}_proj-sceneviewing_task-recognition_run${block}_log-eb_messages_eyetrack.log"
    fi
}


for block_dir in Encoding_Block* Retrieval_Block*; do
    echo "Processing Block: $block_dir"
    block_type=$(echo "$block_dir" | awk -F_ '{print $1}')
    
    for id_folder in "$block_dir"/*; do
        id=$(basename "$id_folder")

        for session_folder in "$id_folder"/*; do
            session_name=$(basename "$session_folder")

            if [[ $session_name =~ ^[0-9]{8}$ ]]; then
                continue
            fi

            date=$(extract_date "$(find "$session_folder" -type f -name "*.edf")")
            if [ $? -ne 0 ]; then
                read -p "Enter date (YYYYMMDD) for $session_folder: " date
            fi

            block=$(extract_block_number "$(find "$session_folder" -type f -name "*Trial*.dat")" "$block_type")
            if [ -z "$block" ]; then
                read -p "Enter block number for $session_folder: " block
            fi

            new_session_dir="${id_folder}/${date}"
            mv "$session_folder" "$new_session_dir"
            echo "$(basename "$session_folder") -> $(basename "$new_session_dir")"

            rename_files "$block_type" "$id" "$date" "$block" "$new_session_dir"
        done
    done
done
